In [1]:
import sys
import os

# Get the current working directory (where the notebook is running)
SCRIPT_DIR = os.getcwd()  # or use os.path.dirname(os.path.abspath('')) for similar effect
sys.path.append(os.path.dirname(SCRIPT_DIR))


# Basic
import numpy as np
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)
import matplotlib.pyplot as plt

# Dataset
from keras.datasets import mnist

# Evaluation
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Your SOM model
from som import SOM


2025-04-28 15:04:49.298077: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 15:04:50.990283: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 15:04:51.755013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745867092.527492    3935 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745867092.701282    3935 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745867094.395903    3935 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten images from (28,28) to (784,)
X_train = X_train.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (60000, 784)
Test shape: (10000, 784)


In [ ]:
# Initialize SOM
som = SOM(
    lattice_rows=5,
    lattice_columns=5,
    initial_learning_rate=0.5,
    neighbourhood_radius=5,
    max_iters=100,
    learning_rate_type="exponential",
    random_state=42,
    verbose=True,
)

# Fit SOM
som.fit(X_train)


In [ ]:
# Predict Best Matching Units (BMUs) for train and test
train_bmus = som.predict(X_train)
test_bmus = som.predict(X_test)

print("Train BMUs shape:", train_bmus.shape)
print("Test BMUs shape:", test_bmus.shape)


In [ ]:
# As SOM gives (row, col), we can map it to a single integer cluster ID
def bmu_to_cluster_id(bmus, n_cols):
    return np.array([row * n_cols + col for row, col in bmus])

train_clusters = bmu_to_cluster_id(train_bmus, som.lattice_columns)
test_clusters = bmu_to_cluster_id(test_bmus, som.lattice_columns)

# Evaluate
print("Train Adjusted Rand Index:", adjusted_rand_score(y_train, train_clusters))
print("Test Adjusted Rand Index:", adjusted_rand_score(y_test, test_clusters))

print("Train Normalized Mutual Information:", normalized_mutual_info_score(y_train, train_clusters))
print("Test Normalized Mutual Information:", normalized_mutual_info_score(y_test, test_clusters))
